In [1]:
####################################
#ENVIRONMENT SETUP

In [2]:
#Importing Libraries
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
import matplotlib.ticker as ticker
import matplotlib.cm as cm
from matplotlib.colors import Normalize
from matplotlib.ticker import MaxNLocator
from matplotlib.ticker import ScalarFormatter
import matplotlib.gridspec as gridspec
import xarray as xr

import sys; import os; import time; from datetime import timedelta
import pickle
import h5py

In [3]:
#MAIN DIRECTORIES
def GetDirectories():
    mainDirectory='/mnt/lustre/koa/koastore/torri_group/air_directory/Projects/DCI-Project/'
    mainCodeDirectory=os.path.join(mainDirectory,"Code/CodeFiles/")
    scratchDirectory='/mnt/lustre/koa/scratch/air673/'
    codeDirectory=os.getcwd()
    return mainDirectory,mainCodeDirectory,scratchDirectory,codeDirectory

[mainDirectory,mainCodeDirectory,scratchDirectory,codeDirectory] = GetDirectories()

In [4]:
#IMPORT CLASSES (from current directory)
sys.path.append(os.path.join(mainCodeDirectory,"Variable_Calculation"))
from CLASSES_Variable_Calculation import ModelData_Class, SlurmJobArray_Class, DataManager_Class

In [5]:
#IMPORT CLASSES (from current directory)
sys.path.append(os.path.join(mainCodeDirectory,"Variable_Calculation"))
from FUNCTIONS_Variable_Calculation import *

In [6]:
####################################
#LOADING CLASSES

In [7]:
#data loading class
ModelData = ModelData_Class(mainDirectory, scratchDirectory, simulationNumber=1)
#data manager class
DataManager = DataManager_Class(mainDirectory, scratchDirectory, ModelData.res, ModelData.t_res, ModelData.Nz_str,
                                ModelData.Np_str, dataType="EntrainmentCalculation", dataName="EntrainmentCalculation",
                                dtype='float32')

=== CM1 Data Summary ===
 Simulation #:   1
 Resolution:     1km
 Time step:      5min
 Vertical levels:34
 Parcels:        1e6
 Data file:      /mnt/lustre/koa/koastore/torri_group/air_directory/Projects/DCI-Project/Model/cm1r20.3/run/cm1out_1km_5min_34nz.nc
 Parcel file:    /mnt/lustre/koa/koastore/torri_group/air_directory/Projects/DCI-Project/Model/cm1r20.3/run/cm1out_pdata_1km_5min_1e6np.nc
 Time steps:     133

=== DataManager Summary ===
 inputDirectory #:   /mnt/lustre/koa/koastore/torri_group/air_directory/Projects/DCI-Project/Code/OUTPUT/Variable_Calculation/TimeSplitModelData
 outputDirectory #:   /mnt/lustre/koa/koastore/torri_group/air_directory/Projects/DCI-Project/Code/OUTPUT/Variable_Calculation/EntrainmentCalculation
 inputDataDirectory #:   /mnt/lustre/koa/koastore/torri_group/air_directory/Projects/DCI-Project/Code/OUTPUT/Variable_Calculation/TimeSplitModelData/1km_5min_34nz/ModelData
 inputParcelDirectory #:   /mnt/lustre/koa/koastore/torri_group/air_directory/Proje

In [8]:
#JOB ARRAY SETUP
UsingJobArray=True

def GetNumJobs(res):
    if res=='1km':
        num_jobs=20
    elif res=='250m': 
        num_jobs=100
    return num_jobs
num_jobs = GetNumJobs(ModelData.res)
SlurmJobArray = SlurmJobArray_Class(total_elements=ModelData.Ntime, num_jobs=num_jobs, UsingJobArray=UsingJobArray)
start_job = SlurmJobArray.start_job; end_job = SlurmJobArray.end_job

def GetNumElements():
    num_elements = np.arange(ModelData.Ntime)[start_job:end_job]
    return num_elements
num_elements = GetNumElements()

Running timesteps from 0:7 



In [9]:
####################################
#FUNCTIONS

In [10]:
def MakeDataDictionary(variableNames,t):
    timeString = ModelData.timeStrings[t]
    # print(f"Getting data from {timeString}","\n")
    
    dataDictionary = {variableName: CallLagrangianArray(ModelData, DataManager, timeString, variableName=variableName) 
                      for variableName in variableNames}      
    return dataDictionary
    
def GetSpatialData(t):    
    variableNames = ['Z', 'Y', 'X']
    dataDictionary = MakeDataDictionary(variableNames,t)
    Z,Y,X = (dataDictionary[k] for k in variableNames)
    return Z,Y,X

def GetAData(t,PROCESSING):
    if PROCESSING==False:    
        variableNames = ['A_g', 'A_c']
    elif PROCESSING==True: #*#*
        variableNames = ['A_g_Processed', 'A_c_Processed'] #*#*

    dataDictionary = MakeDataDictionary(variableNames,t)
    A_g,A_c = (dataDictionary[k] for k in variableNames)
    return A_g,A_c

In [11]:
def SubtractA(A,A_Prior):
    D = np.zeros_like(A,dtype=np.int8)
    D = A*1 - A_Prior*1
    return D

def AddAs(A1,A2_Prior):
    S = np.zeros_like(A1,dtype=np.int8)
    S = A1*1 + A2_Prior*1
    return S

In [12]:
def CalculateEntrainment(t, Z,Y,X, A1,A2, A1_Prior,A2_Prior):
    """
    Function to compute 3D entrainment and update result array based on provided inputs.
    
    Returns a 3D (t,z) array containing the sum of the D array representing entrained parcels, by 1, and detrained parcels, by -1.
    The finally array is then ordered by the appropiate index using the np.add.at function
    
    Parameters:
    - A: The (t,p) lagrangian binary array.
    - T: The (t,p) lagrangian time index array.
    - Z: The (t,p) Lagrangian z index array.
    - Y: The (t,p) Lagrangian y index array.
    - X: The (t,p) Lagrangian x index array.

    """
    #Calculation for Entrainment and Detrainment
    DMatrix_Entrainment = SubtractA(A2,A2_Prior)
    DMatrix_Detrainment = DMatrix_Entrainment.copy()

    # Update D for entrainment/detrainment
    DMatrix_Entrainment[DMatrix_Entrainment < 0] = 0
    DMatrix_Detrainment[DMatrix_Detrainment > 0] = 0
    
    #General <==> Cloudy Updraft-Transfer Entrainment
    SMatrix_Entrainment = AddAs(A1,A2_Prior) 
    SMatrix_Detrainment = AddAs(A2,A1_Prior)
        
    #FINDING LOCATIONS WITH TRANSFER ENTRAINMENT
    SMatrix_Entrainment = (SMatrix_Entrainment==2)*1
    SMatrix_Detrainment = (SMatrix_Detrainment==2)*1
    
    # #Removing Transfer-Entrainment where D==0 (just in case)
    SMatrix_Entrainment[DMatrix_Detrainment==0]=0 
    SMatrix_Detrainment[DMatrix_Entrainment==0]=0 
    
    # Initialize time and vertical dimension arrays
    Nz = ModelData.Nzh; Ny = ModelData.Nyh; Nx = ModelData.Nxh
    
    # Initialize result array
    Entrainment = np.zeros((Nz, Ny, Nx),dtype="int32")
    Detrainment = Entrainment.copy()
    TransferEntrainment = Entrainment.copy()
    TransferDetrainment = Entrainment.copy()

    if t==0:
        return Entrainment,Detrainment, TransferEntrainment,TransferDetrainment
    else:
        # Use np.add.at to accumulate values in the result array
        np.add.at(Entrainment, (Z, Y, X), DMatrix_Entrainment)
        np.add.at(Detrainment, (Z, Y, X), DMatrix_Detrainment)
    
        np.add.at(TransferEntrainment, (Z, Y, X), SMatrix_Entrainment)
        np.add.at(TransferDetrainment, (Z, Y, X), SMatrix_Detrainment)

        return Entrainment,Detrainment, TransferEntrainment,TransferDetrainment

In [13]:
def RunCalculation(t, Z,Y,X, A_g,A_c, A_g_Prior,A_c_Prior): #*#* 

    [Entrainment_g, Detrainment_g,
     TransferEntrainment_c_to_g,TransferDetrainment_c_to_g] = CalculateEntrainment(t, Z,Y,X, A1=A_c,A2=A_g, A1_Prior=A_c_Prior,A2_Prior=A_g_Prior)

    [Entrainment_c, Detrainment_c,
     TransferEntrainment_g_to_c,TransferDetrainment_g_to_c] = CalculateEntrainment(t, Z,Y,X, A1=A_g,A2=A_c, A1_Prior=A_g_Prior,A2_Prior=A_c_Prior)

    outputDictionary_Entrainment = {
        "Entrainment_g": Entrainment_g,
        "TransferEntrainment_c_to_g": TransferEntrainment_c_to_g,
        "Entrainment_c": Entrainment_c,
        "TransferEntrainment_g_to_c": TransferEntrainment_g_to_c,
        }
    
    outputDictionary_Detrainment = {
        "Detrainment_g": Detrainment_g,
        "TransferDetrainment_c_to_g": TransferDetrainment_c_to_g,
        "Detrainment_c": Detrainment_c,
        "TransferDetrainment_g_to_c": TransferDetrainment_g_to_c
        }

    return outputDictionary_Entrainment, outputDictionary_Detrainment

In [14]:
##############################################
#RUNNING
PROCESSING=False #set to True if using Turbulence-Removed Binary Arrays
# PROCESSING=True

In [15]:
#CALCULATING AND APPENDING TO DATA EACH TIMESTEP
for t in num_elements:
    if np.mod(t,1)==0: print(f'Current time {t}')

    #loading data
    [Z,Y,X] = GetSpatialData(t)
    [A_g,A_c] = GetAData(t,PROCESSING) 
    [A_g_Prior,A_c_Prior] = GetAData(t-1,PROCESSING) 

    #calculating variables
    [outputDictionary_Entrainment,outputDictionaryDetrainment] = RunCalculation(t, Z,Y,X, A_g,A_c, A_g_Prior,A_c_Prior)
    
    #outputting
    timeString = ModelData.timeStrings[t]
    timeString_Prior = ModelData.timeStrings[t-1]
    
    DataManager.SaveOutputTimestep(DataManager.outputDataDirectory, timeString, outputDictionary_Entrainment, dataName="Entrainment")
    DataManager.SaveOutputTimestep(DataManager.outputDataDirectory, timeString_Prior, outputDictionaryDetrainment, dataName="Detrainment")

    break

Current time 0
Saved timestep to output file: /mnt/lustre/koa/koastore/torri_group/air_directory/Projects/DCI-Project/Code/OUTPUT/Variable_Calculation/EntrainmentCalculation/1km_5min_34nz/EntrainmentCalculation/Entrainment_1km_5min_34nz_0-00-00.h5 

Saved timestep to output file: /mnt/lustre/koa/koastore/torri_group/air_directory/Projects/DCI-Project/Code/OUTPUT/Variable_Calculation/EntrainmentCalculation/1km_5min_34nz/EntrainmentCalculation/Detrainment_1km_5min_34nz_11-00-00.h5 

